# CODEtect Algorithm

## Introduction
CODEtect algorithm exploits the graph representation of financial transactions in the accounting information system and aims to identify anomalous financial transactions. It is framed as a problem of detecting anomalous graphs within a large database containing graphs with labeled nodes and directed multi-edges. 

The core idea is to explain a series of financial transactions represented as graphs by common patterns. The algorithm identifies a small representative set of structural patterns (i.e., node-labeled network motifs) that losslessly compress the database as concisely as possible. Financial transactions that do not compress well are flagged as anomalous. In our case, the common patterns are described by motifs (recurrent and statistically significant subgraphs of a larger graph), and each motif is associated with a number called "code length." The more common the motif, the shorter the code length. Hence, a normal series of financial transactions should be able to be decomposed into a handful of common motifs; thus, this series of transactions will have a short code length. On the other hand, an anomaly series of transactions will have a long code length, as it cannot be explained by common motifs (motifs with short code lengths.)

This Jupyter notebook shows an example code using this algorithm to detect anomalous financial transactions.


## Dependency
First, we install the corresponding packages.

In [4]:
# Install packages
!pip install networkx[default]
!pip install pandas
!pip install tqdm

import os
import pickle
import numpy as np
import networkx as nx
import pandas as pd
import tqdm
from math import log
from scipy.special import perm

     |████████████████████████████████| 55.6 MB 49.9 MB/s eta 0:00:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


## Load existing motifs
Network motifs are recurrent and statistically significant subgraphs of a larger graph. Simply put, a motif is a graph pattern that occurs frequently. We provide the motif information extracted from the large datasets, where many graphs are first considered, and the most common patterns (motifs) are found using greedy algorithms. The common motifs are then assigned code lengths -- the more common the motif occurs in the large datasets, the shorter its code length is.

In [5]:
with open('all_output_0.pkl', "rb") as f:
    (M_table, M2_table, graphs_code_length, cover_sets, M_usages, M_code_table) = pickle.load(f)
with open('all_scores_0.pkl', "rb") as f:
    (Gid, graphs_code_length) = pickle.load(f)
with open('all_motifs_0.pkl', "rb") as f:
    all_motifs = pickle.load(f)

In [6]:
with open('motif.pkl', "rb") as f:
    M_candidate = pickle.load(f)
with open('occurrence2motif.pkl', "rb") as f:
    occ_M = pickle.load(f)

## Create the motif table
Then we create the motif table, where each motif corresponds to a code length.
Which patterns (motifs) to include in the code tables?-->build a model for the "norm"
Simple answer: Minimum Description Length

In [7]:
# parse and create motif tables for length 2 and 3
# motif_2: the motif table of length 2: dict of {(type1, type2): score}
# motif_3: the motif table of length 3: list of ({type1: [out_count, in_count], type2: [out_count, in_count]}, score)
motif_2 = []
motif_3 = []
for (l, score) in M_code_table.items():
    if isinstance(l, int):
        G = M_candidate[l]
        edges_by_nodes = {}
        all_nodes = {a:b["type"] for (a,b) in list(G.nodes(data=True))}
        transaction_node = -1
        for (a, b) in all_nodes.items():
            if b == "Other":
                transaction_node = a
                break
        assert(transaction_node != -1)
        all_edges = [(a,b,int(c["weight"])) for (a,b,c) in list(G.edges(data=True))]
        for (a,b,w) in all_edges:
            if a == transaction_node:
                node_type = all_nodes[b]
                edges_by_nodes[node_type] = edges_by_nodes.get(node_type, np.array([0,0])) + np.array([0,w])
            elif b == transaction_node:
                node_type = all_nodes[a]
                edges_by_nodes[node_type] = edges_by_nodes.get(node_type, np.array([0,0])) + np.array([w,0])
            else:
                assert(False)
        motif_3.append((edges_by_nodes, score))
    else:
        motif_2.append((l, score))

motif_2 = {a:b for (a,b) in motif_2}
motif_3 = sorted(motif_3, key = lambda x:x[1])

## Check the motif table
Now we take a look at the motif table.

In [10]:
pd.Series(motif_2)

Short-term Financial Liabilities  Other                               14.990628
Operating Revenue/Sales           Other                                5.346372
Operating Expenses                Other                                1.697660
Long-term Financial Liabilities   Other                               18.160553
Long-term Operating Assets        Other                               16.575591
Short-term Operating Assets       Other                                8.730101
Non-operating Gains or losses     Other                                7.702147
Long-term Operating Liabilities   Other                               15.838625
Short-term Operating Liabilities  Other                                7.232035
Cash                              Other                               13.302572
Other                             Short-term Financial Liabilities    15.575591
                                  Operating Revenue/Sales              5.356120
                                  Operat

Note: As you can see, the motif table consists of graphs with two only vertices (or we call nodes). (The omitted entries are Other). The first column and the second column correspond to two nodes in the graphs (two accounting classifications). The numbers indicate various code lengths.

Some helper functions to combine graphs:

In [11]:
def multi_to_simple(M):
    edge_count={}
    G = nx.DiGraph()
    for u,v,data in M.edges(data=True):
        w = data['weight'] if 'weight' in data else 1.0
        if G.has_edge(u,v):
            G[u][v]['weight'] += w
            edge_count[(u,v)]+=1
        else:
            G.add_edge(u, v, weight=w)
            edge_count[(u,v)]=1

    return G,edge_count

def div_array(arr_a, arr_b):
    res_0 = 1e5 if arr_b[0] == 0 else arr_a[0] // arr_b[0]
    res_1 = 1e5 if arr_b[1] == 0 else arr_a[1] // arr_b[1]
    return min(res_0, res_1)

def encode_integer(n):
    if n <= 0 :
        return 0
    c = log(2.865064,2);
    i = log(n,2);
    while i > 0 :
        c = c + i
        i = log(i,2)
    return c

Then we load the account type information from another file:

In [12]:
account_address = "account_types_new.pkl"
account_types = pickle.load(open(account_address, 'rb'))
type_map=sorted(set(account_types.values()))
account_numbers=set(account_types.keys())
account_types['0'] = "Other"

## Detect Anomalies
Now we have all the motif information, we can detect anomalies! 

We first load the series of transactions we want to analyze:

In [13]:
Gid = []
all_code_length = []
file = "top1.csv"
curr_df=pd.read_csv(file, dtype=object, index_col=0)
curr_df

,Source,Destination,Amount
Journal_ID,,,
400009726,0,1000000433,122055.120000
400009726,0,1000000434,5530586.510000
400009726,0,1000000741,772528.070000
400009726,3000000000,0,443766.000000
400009726,0,3000000051,909.000000
...,...,...,...
400009726,0,1000003878,2024331.940000
400009726,0,1000003878,14625.000000
400009726,0,2000000427,113.160000


We then run the algorithm to analyze its code length:

In [14]:
try:
    jid=int(file.split('.')[0])
except ValueError:
    jid=str(file.split('.')[0])
M=nx.MultiDiGraph()
sources=curr_df.Source
destinations=curr_df.Destination
edges=list(zip(sources,destinations))
M.add_edges_from(edges)
#G,edge_counts=multi_to_simple(M)
G = M
nx.set_node_attributes(G,account_types,'type')
# add 0 as Other, no need since 0 is already in account_type.pkl
# nx.set_node_attributes(G,{'0': "Other"},'type')
# check for accounts not present in account_type.pkl
if len(nx.get_node_attributes(G, "type")) != len(G.nodes(data=True)):
    assert(False)
Gid.append(jid)
G_size = len(list(G.nodes(data=True)))

# generate edge_by_nodes for each graph, same formate as in motif_3
edges_by_nodes = {}
all_nodes = {a:b["type"] for (a,b) in list(G.nodes(data=True))}
transaction_node = -1
for (a, b) in all_nodes.items():
    if b == "Other":
        transaction_node = a
        break
assert(transaction_node != -1)
all_edges = [(a,b) for (a,b,c) in list(G.edges(data=True))]
#print(all_edges)
for (a,b) in all_edges:
    if a == transaction_node:
        node_type = all_nodes[b]
        edges_by_nodes[node_type] = edges_by_nodes.get(node_type, np.array([0,0])) + np.array([0,1])
    elif b == transaction_node:
        node_type = all_nodes[a]
        edges_by_nodes[node_type] = edges_by_nodes.get(node_type, np.array([0,0])) + np.array([1,0])
    else:
        assert(False)
# print(edges_by_nodes)

# greedily cover G using the motif table, start with most frequent one in motif_3
code_length = 0
for (motif_edge_list, score) in motif_3:
    motif_size = len(motif_edge_list)
    num_copies = 1e5
    #print(motif_edge_list)
    for (node_type, motif_out_in) in motif_edge_list.items():
        # corresponding type in G
        G_out_in = edges_by_nodes.get(node_type, np.array([0,0]))
        #print(G_out_in, motif_out_in)
        num_copies = min(num_copies, div_array(G_out_in, motif_out_in))
    assert(num_copies != 1e5)
    if num_copies > 0:
        # covered by current motif for num_copies times
        #print(G_size, motif_size)
        code_length += score + log(perm(G_size, motif_size),2) + encode_integer(num_copies)
        # remove current motif * num_copies from G's edges_by_nodes
        for (node_type, motif_out_in) in motif_edge_list.items():
            edges_by_nodes[node_type] -= num_copies * motif_out_in
            assert(edges_by_nodes[node_type][0] >= 0 and edges_by_nodes[node_type][1] >= 0)
        print("Motif_3:")
        #print(motif_edge_list)
        for (node_type, motif_out_in) in motif_edge_list.items():
            if motif_out_in[0] > 0:
                print("%s --> Other, weight = %d" % (node_type, motif_out_in[0]))
            if motif_out_in[1] > 0:
                print("Other --> %s, weight = %d" % (node_type, motif_out_in[1]))
        print("Total code length = %.3f" % (score + log(perm(G_size, motif_size),2) + encode_integer(num_copies)))
        print("Number of copies = %d" % num_copies)
        print()

# finish covering with motif_2
for (node_type, G_out_in) in edges_by_nodes.items():
    # node_type --> other
    if G_out_in[0] > 0:
        num_copies = G_out_in[0]
        score = motif_2[(node_type, "Other")]
        code_length += score + log(perm(G_size, 2),2) + encode_integer(num_copies)
        print("Motif_2 : (%s, %s)" % (node_type, "Other"))
        print("Total code length = %.3f" % (score + log(perm(G_size, 2),2) + encode_integer(num_copies)))
        print("Number of copies = %d" % num_copies)
        print()
    if G_out_in[1] > 0:
    # other --> node_type
        num_copies = G_out_in[1]
        score = motif_2[("Other", node_type)]
        code_length += score + log(perm(G_size, 2),2) + encode_integer(num_copies)
        print("Motif_2 : (%s, %s)" % ("Other", node_type))
        print("Total code length = %.3f" % (score + log(perm(G_size, 2),2) + encode_integer(num_copies)))
        print("Number of copies = %d" % num_copies)
        print()
print("id = %s, total score = %.3f" % (jid, code_length))
all_code_length.append((jid, code_length))
#assert(False)

print("Count = %d" % len(all_code_length))

Motif_3:
Operating Expenses --> Other, weight = 6
Other --> Short-term Operating Liabilities, weight = 6
Total code length = 20.082
Number of copies = 1

Motif_3:
Other --> Short-term Operating Liabilities, weight = 2
Cash --> Other, weight = 2
Total code length = 21.397
Number of copies = 1

Motif_3:
Inter-Company Liabilities --> Other, weight = 2
Other --> Operating Expenses, weight = 2
Total code length = 23.906
Number of copies = 1

Motif_3:
Other --> Short-term Operating Assets, weight = 2
Non-operating Gains or losses --> Other, weight = 2
Total code length = 24.742
Number of copies = 1

Motif_3:
Other --> Operating Expenses, weight = 2
Contributed Equity --> Other, weight = 2
Total code length = 33.450
Number of copies = 11

Motif_3:
Operating Expenses --> Other, weight = 2
Other --> Contributed Equity, weight = 2
Total code length = 28.359
Number of copies = 2

Motif_3:
Contributed Equity --> Other, weight = 2
Other --> Contributed Equity, weight = 2
Total code length = 24.942


As we can see, we break down the transaction graphs into multiple copies of different motifs. Since this series of transactions have many copies of high code length motifs, it contains patterns that "normal" transaction patterns cannot explain. Therefore, we regard it as highly anomalous. In reality, we have run this algorithm on millions of transactions to find transactions with high code lengths, then analyze the top ones with high code lengths by experts.